**Full Factorial vs. Isolation Approach: Overview**

The full factorial design approach used in this notebook differs significantly from an isolation approach in several key ways:

**Full Factorial Approach**

**Tests all combinations**: Instead of testing one factor at a time, the full
factorial design tests all possible combinations of architecture, filter size, depth, and regularization.

**Reveals interaction effects**: By testing all combinations, we can discover how factors interact with each other. For example, we might find that a specific architecture works particularly well with a certain filter size, which wouldn't be apparent when testing in isolation.

**Finds the true optimum**: The factorial approach is more likely to find the globally optimal configuration because it explores the entire parameter space rather than optimizing each parameter independently.

**Statistical analysis**: Enables formal analysis of main effects and interactions, providing quantitative evidence of each factor's impact.

**Comprehensive but resource-intensive**: Requires more computational resources as the number of combinations grows exponentially with the number of factors and levels.

**Isolation Approach (Used in Previous Experiments)**

**Tests one factor at a time**: Changes only one parameter while keeping others constant.

**Misses interactions**: Cannot detect how different factors might work together or against each other.

**May find local optima**: By optimizing each parameter separately, you might end up with a suboptimal configuration.

**More efficient**: Requires fewer experiments, making it less computationally expensive.

**Simpler analysis**: Results are easier to interpret but provide less insight into complex relationships.

The full factorial approach in this notebook provides a more thorough understanding of how different CNN configurations affect waste classification performance, allowing us to answer both research questions with greater confidence and detail. It reveals not just which individual settings work best, but how they work together as a complete system.





## Step 1: Importing the necessary libaries


In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import product
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from IPython.display import display, Markdown
from google.colab import drive

# MOUNT GOOGLE DRIVE
drive.mount('/content/drive')
Files_save_path = '/content/drive/MyDrive/<path>'
os.makedirs(Files_save_path, exist_ok=True)

## Step 2: Load the Dataset into TensorFlow


In [ ]:
# PATH TO THE DATASET
dataset_path = '/content/drive/MyDrive/<path to dataset>'

# TRAINING DATASET V2 (80% OF DATA)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

# VALIDATION DATASET V2 (20% OF DATA)
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=34
)

# CLASS NAMES
class_names = train_ds.class_names
print(f"Class Names: {class_names}")

## Step 3: Visualizing the images



In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # IT WILL TAKE ONE BATCH
    for i in range(9):  # SHOW 9 IMAGES
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # CONVERT TO PROPER FORMAT
        plt.title(class_names[labels[i]])  # ADD LABEL
        plt.axis("off")
    break
plt.show()

## Step 4: Normalize the data


In [ ]:
# NORMALIZE PIXEL VALUES (DIVIDE BY 255)
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))  # X REPRESENTS IMAGE DATA, Y REPRESENTS LABELS
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

# CONFIGURING THE DATASET FOR PERFORMANCE
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Step 5: Defining the Full Factorial Experiment Parameters

**Purpose**: Set up all possible combinations of CNN configurations to test

**Why**: This systematic approach ensures we find the optimal configuration by testing all interactions between factors




In [ ]:
# DEFINING THE FACTORS AND THEIR LEVELS
architectures = ['Simple', 'VGG', 'ResNet', 'MobileNet']
filter_sizes = [(3, 3), (5, 5), (7, 7)]
depths = [2, 3, 4]  # REDUCED FROM [2,3,4,5] TO LIMIT TOTAL COMBINATIONS
regularizations = ['None', 'Dropout', 'L2', 'BatchNorm']

# CREATING ALL POSSIBLE COMBINATIONS
all_combinations = list(product(architectures, filter_sizes, depths, regularizations))
print(f"Total number of combinations to test: {len(all_combinations)}")
print(f"First 5 combinations: {all_combinations[:5]}")

# CREATING A DIRECTORY TO SAVE MODELS
models_save_path = '/content/drive/MyDrive/<path>'
os.makedirs(models_save_path, exist_ok=True)

## Step 6: Creating the Model Factory Function


**Purpose**: Generate CNN models based on specific combinations of architecture, filter size, depth, and regularization

**Why**: This modular approach allows us to systematically test all parameter combinations



In [ ]:
def create_model_with_combination(architecture, filter_size, depth, regularization):
    """CREATE A MODEL WITH THE SPECIFIED COMBINATION OF FACTORS"""

    # BASE PARAMETERS
    reg_value = 0.2 if regularization == 'Dropout' else 0.001 if regularization == 'L2' else None

    #1.  SIMPLE CNN IMPLEMENTATION
    if architecture == 'Simple':
        model = tf.keras.Sequential()

        # INPUT LAYER
        kernel_reg = tf.keras.regularizers.l2(reg_value) if regularization == 'L2' else None
        model.add(tf.keras.layers.Conv2D(32, filter_size, activation='relu',
                                         kernel_regularizer=kernel_reg,
                                         input_shape=(128, 128, 3)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        # APPLY REGULARIZATION IF SPECIFIED
        if regularization == 'Dropout':
            model.add(tf.keras.layers.Dropout(reg_value))
        elif regularization == 'BatchNorm':
            model.add(tf.keras.layers.BatchNormalization())

        # ADD ADDITIONAL LAYERS BASED ON DEPTH
        filters = 64
        for _ in range(depth - 1):  # -1 BECAUSE WE ALREADY ADDED ONE CONV BLOCK
            model.add(tf.keras.layers.Conv2D(filters, filter_size, activation='relu',
                                            kernel_regularizer=kernel_reg))
            model.add(tf.keras.layers.MaxPooling2D(2, 2))

            # APPLY REGULARIZATION IF SPECIFIED
            if regularization == 'Dropout':
                model.add(tf.keras.layers.Dropout(reg_value))
            elif regularization == 'BatchNorm':
                model.add(tf.keras.layers.BatchNormalization())

            filters = min(filters * 2, 512)  # DOUBLE FILTERS UP TO 512

        # CLASSIFIER
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu',
                                       kernel_regularizer=kernel_reg))

        # APPLY REGULARIZATION IF SPECIFIED
        if regularization == 'Dropout':
            model.add(tf.keras.layers.Dropout(reg_value))
        elif regularization == 'BatchNorm':
            model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))


    #2. VGG-STYLE IMPLEMENTATION
    elif architecture == 'VGG':
        model = tf.keras.Sequential()

        # FIRST BLOCK
        kernel_reg = tf.keras.regularizers.l2(reg_value) if regularization == 'L2' else None
        model.add(tf.keras.layers.Conv2D(64, filter_size, padding='same', activation='relu',
                                        kernel_regularizer=kernel_reg,
                                        input_shape=(128, 128, 3)))

        # ADD MORE BLOCKS BASED ON DEPTH
        filters = 64
        for i in range(depth):
            # SECOND CONV IN THE BLOCK
            model.add(tf.keras.layers.Conv2D(filters, filter_size, padding='same',
                                           activation='relu',
                                           kernel_regularizer=kernel_reg))

            # APPLY REGULARIZATION IF SPECIFIED
            if regularization == 'BatchNorm':
                model.add(tf.keras.layers.BatchNormalization())

            model.add(tf.keras.layers.MaxPooling2D(2, 2))

            # APPLY DROPOUT IF SPECIFIED
            if regularization == 'Dropout':
                model.add(tf.keras.layers.Dropout(reg_value))

            # INCREASE FILTERS FOR NEXT BLOCK
            filters = min(filters * 2, 512)

        # CLASSIFIER
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(512, activation='relu',
                                      kernel_regularizer=kernel_reg))

        # APPLY REGULARIZATION IF SPECIFIED
        if regularization == 'Dropout':
            model.add(tf.keras.layers.Dropout(reg_value))
        elif regularization == 'BatchNorm':
            model.add(tf.keras.layers.BatchNormalization())

        model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))


# 3. RESNET-STYLE IMPLEMENTATION
    elif architecture == 'ResNet':
        inputs = tf.keras.Input(shape=(128, 128, 3))

        # FIRST CONV LAYER
        kernel_reg = tf.keras.regularizers.l2(reg_value) if regularization == 'L2' else None
        x = tf.keras.layers.Conv2D(64, filter_size, strides=2, padding='same',
                                 kernel_regularizer=kernel_reg)(inputs)

        if regularization == 'BatchNorm':
            x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation('relu')(x)
        x = tf.keras.layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

        # ADD RESIDUAL BLOCKS BASED ON DEPTH
        filters = 64
        for i in range(depth):
            # RESIDUAL BLOCK
            shortcut = x

            # FIRST CONV IN BLOCK
            x = tf.keras.layers.Conv2D(filters, filter_size, padding='same',
                                     kernel_regularizer=kernel_reg)(x)

            if regularization == 'BatchNorm':
                x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.Activation('relu')(x)

            # SECOND CONV IN BLOCK
            x = tf.keras.layers.Conv2D(filters, filter_size, padding='same',
                                     kernel_regularizer=kernel_reg)(x)

            if regularization == 'BatchNorm':
                x = tf.keras.layers.BatchNormalization()(x)

            # HANDLE SHORTCUT FOR DIMENSION CHANGES
            if i > 0:  # FOR BLOCKS AFTER THE FIRST ONE
                shortcut = tf.keras.layers.Conv2D(filters, (1, 1), strides=1, padding='same',
                                               kernel_regularizer=kernel_reg)(shortcut)
                if regularization == 'BatchNorm':
                    shortcut = tf.keras.layers.BatchNormalization()(shortcut)

            # ADD THE SHORTCUT TO THE MAIN PATH
            x = tf.keras.layers.add([x, shortcut])
            x = tf.keras.layers.Activation('relu')(x)

            if regularization == 'Dropout':
                x = tf.keras.layers.Dropout(reg_value)(x)

            # INCREASE FILTERS FOR NEXT BLOCK
            filters = min(filters * 2, 512)

        # GLOBAL AVERAGE POOLING AND CLASSIFIER
        x = tf.keras.layers.GlobalAveragePooling2D()(x)

        if regularization == 'Dropout':
            x = tf.keras.layers.Dropout(reg_value)(x)

        x = tf.keras.layers.Dense(len(class_names), activation='softmax',
                                kernel_regularizer=kernel_reg)(x)

        model = tf.keras.Model(inputs, x)


# 4.MOBILENET-STYLE IMPLEMENTATION
    elif architecture == 'MobileNet':
        def depthwise_separable_conv(x, filters, stride=1):
            kernel_reg = tf.keras.regularizers.l2(reg_value) if regularization == 'L2' else None

            x = tf.keras.layers.DepthwiseConv2D(
                kernel_size=filter_size, strides=stride, padding='same',
                kernel_regularizer=kernel_reg)(x)

            if regularization == 'BatchNorm':
                x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.ReLU()(x)

            x = tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same',
                                     kernel_regularizer=kernel_reg)(x)

            if regularization == 'BatchNorm':
                x = tf.keras.layers.BatchNormalization()(x)

            x = tf.keras.layers.ReLU()(x)

            if regularization == 'Dropout':
                x = tf.keras.layers.Dropout(reg_value)(x)

            return x

        inputs = tf.keras.Input(shape=(128, 128, 3))

        # FIRST CONV LAYER
        kernel_reg = tf.keras.regularizers.l2(reg_value) if regularization == 'L2' else None
        x = tf.keras.layers.Conv2D(32, filter_size, strides=2, padding='same',
                                 kernel_regularizer=kernel_reg)(inputs)

        if regularization == 'BatchNorm':
            x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.ReLU()(x)

        filters = 64
        for i in range(depth):
            stride = 2 if i > 0 else 1  # USE STRIDE 2 FOR ALL BLOCKS EXCEPT THE FIRST
            x = depthwise_separable_conv(x, filters, stride=stride)
            filters = min(filters * 2, 512)  # DOUBLE FILTERS UP TO 512

        # GLOBAL AVERAGE POOLING AND CLASSIFIER
        x = tf.keras.layers.GlobalAveragePooling2D()(x)

        if regularization == 'Dropout':
            x = tf.keras.layers.Dropout(reg_value)(x)

        x = tf.keras.layers.Dense(len(class_names), activation='softmax',
                                kernel_regularizer=kernel_reg)(x)

        model = tf.keras.Model(inputs, x)

    return model

## Step 7: Defining Training and Evaluation Function

**Purpose**: Create a standardized function to train and evaluate each model configuration

**Why**: Ensures consistent training procedures across all models, isolating architecture effects



In [ ]:
def train_and_evaluate_combination(combination, epochs=15):
    """TRAINING AND EVALUATING A MODEL WITH THE SPECIFIED COMBINATION OF FACTORS"""
    architecture, filter_size, depth, regularization = combination

    # CREATE A DESCRIPTIVE NAME FOR THIS COMBINATION
    model_name = f"{architecture}-Filter{filter_size[0]}x{filter_size[1]}-Depth{depth}-{regularization}"
    print(f"\n=== Training {model_name} ===")

    # CREATE THE MODEL
    model = create_model_with_combination(architecture, filter_size, depth, regularization)

    # COMPILE THE MODEL
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # DISPLAY MODEL SUMMARY
    model.summary()

    # TRAIN THE MODEL
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=5, restore_best_weights=True
            )
        ]
    )

    # EVALUATE ON VALIDATION SET
    val_loss, val_acc = model.evaluate(val_ds)
    print(f"Validation accuracy: {val_acc:.4f}")

    # SAVE RESULTS
    results = {
        'combination': combination,
        'model_name': model_name,
        'architecture': architecture,
        'filter_size': f"{filter_size[0]}x{filter_size[1]}",
        'depth': depth,
        'regularization': regularization,
        'val_accuracy': val_acc,
        'val_loss': val_loss,
        'history': history.history,
        'model': model
    }

    # SAVE MODEL
    model_filename = f'waste_model_{model_name.lower().replace(" ", "_").replace("-", "_")}.keras'
    model_path = os.path.join(models_save_path, model_filename)
    model.save(model_path)
    print(f"Model saved to {model_path}")

    return results

## Step 8: Running the Full Factorial Experiment

**Purpose**: Execute the experiment with all combinations of parameters

**Why**: This comprehensive approach reveals interactions between factors that individual tests would miss




In [ ]:
def run_full_factorial_experiment(combinations, max_combinations=None):
    """RUN THE FULL FACTORIAL EXPERIMENT WITH THE SPECIFIED COMBINATIONS"""
    # LIMIT THE NUMBER OF COMBINATIONS IF SPECIFIED
    if max_combinations is not None and max_combinations < len(combinations):
        print(f"Limiting to {max_combinations} combinations out of {len(combinations)}")
        combinations = combinations[:max_combinations]

    # STORE RESULTS
    all_results = []

    # TRAIN AND EVALUATE EACH COMBINATION
    for i, combination in enumerate(combinations):
        print(f"\nCombination {i+1}/{len(combinations)}")
        result = train_and_evaluate_combination(combination)
        all_results.append(result)

        # CLEAR MEMORY
        tf.keras.backend.clear_session()

    return all_results

max_combinations = 100  # ADJUSTMENT BASED ON COMPUTATIONAL RESOURCES AND TIME CONSTRAINTS
factorial_results = run_full_factorial_experiment(all_combinations, max_combinations)

# SAVE THE RESULTS TO DISK FOR LATER ANALYSIS
import pickle
with open(os.path.join(Files_save_path, 'factorial_results.pkl'), 'wb') as f:
    # REMOVE THE ACTUAL MODEL OBJECTS BEFORE SAVING TO REDUCE FILE SIZE
    results_to_save = [{k: v for k, v in r.items() if k != 'model'} for r in factorial_results]
    pickle.dump(results_to_save, f)

print(f"Results saved to {os.path.join(Files_save_path, 'factorial_results.pkl')}")

# Step 9: Analyze Results and Find the Best Configuration


**Purpose**: Process experimental results to identify optimal CNN configurations

**Why**: Helps answer research question 3a by finding the best overall configuration




In [ ]:
def analyze_factorial_results(results):
    """ANALYZE THE RESULTS OF THE FACTORIAL EXPERIMENT"""
    # CREATE A DATAFRAME FOR EASIER ANALYSIS
    results_df = pd.DataFrame([
        {
            'Model Name': r['model_name'],
            'Architecture': r['architecture'],
            'Filter Size': r['filter_size'],
            'Depth': r['depth'],
            'Regularization': r['regularization'],
            'Validation Accuracy': r['val_accuracy'],
            'Validation Loss': r['val_loss'],
            'Training Accuracy': max(r['history']['accuracy']) if 'accuracy' in r['history'] else 0,
            'Accuracy Gap': max(r['history']['accuracy']) - r['val_accuracy'] if 'accuracy' in r['history'] else 0
        }
        for r in results
    ])

    # SORT BY VALIDATION ACCURACY
    results_df = results_df.sort_values('Validation Accuracy', ascending=False)

    # DISPLAY THE TOP 10 CONFIGURATIONS
    print("Top 10 Configurations by Validation Accuracy:")
    display(results_df.head(10))

    # FIND THE BEST CONFIGURATION
    best_config = results_df.iloc[0]
    print(f"\nBest Configuration:")
    print(f"Model: {best_config['Model Name']}")
    print(f"Validation Accuracy: {best_config['Validation Accuracy']:.4f}")

    return results_df

# ANALYZE THE RESULTS
results_df = analyze_factorial_results(factorial_results)

# SAVE THE RESULTS DATAFRAME TO CSV
results_df.to_csv(os.path.join(Files_save_path, 'factorial_results_summary.csv'), index=False)
print(f"Results summary saved to {os.path.join(Files_save_path, 'factorial_results_summary.csv')}")

# Step 10: Visualizing the Impact of Different Factors


**Purpose**: Create visualizations to understand how each factor affects model performance

**Why**: Helps answer research question 3b by showing the impact of different configurations



In [ ]:
# 1. IMPACT OF ARCHITECTURE
plt.figure(figsize=(12, 6))
arch_data = results_df.groupby('Architecture')['Validation Accuracy'].mean().reset_index()
arch_data = arch_data.sort_values('Validation Accuracy', ascending=False)

plt.bar(arch_data['Architecture'], arch_data['Validation Accuracy'], color='royalblue')
plt.title('Average Validation Accuracy by Architecture')
plt.xlabel('Architecture')
plt.ylabel('Average Validation Accuracy')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# ADD VALUE LABELS
for i, v in enumerate(arch_data['Validation Accuracy']):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'architecture_impact.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'architecture_impact.pdf'), dpi=300)
plt.show()

# 2. IMPACT OF FILTER SIZE
plt.figure(figsize=(12, 6))
filter_data = results_df.groupby('Filter Size')['Validation Accuracy'].mean().reset_index()
filter_data = filter_data.sort_values('Validation Accuracy', ascending=False)

plt.bar(filter_data['Filter Size'], filter_data['Validation Accuracy'], color='forestgreen')
plt.title('Average Validation Accuracy by Filter Size')
plt.xlabel('Filter Size')
plt.ylabel('Average Validation Accuracy')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# ADD VALUE LABELS
for i, v in enumerate(filter_data['Validation Accuracy']):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'filter_size_impact.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'filter_size_impact.pdf'), dpi=300)
plt.show()

# 3. IMPACT OF NETWORK DEPTH
plt.figure(figsize=(12, 6))
depth_data = results_df.groupby('Depth')['Validation Accuracy'].mean().reset_index()
depth_data = depth_data.sort_values('Depth')  # SORT BY DEPTH FOR BETTER VISUALIZATION

plt.bar(depth_data['Depth'].astype(str), depth_data['Validation Accuracy'], color='darkorange')
plt.title('Average Validation Accuracy by Network Depth')
plt.xlabel('Network Depth')
plt.ylabel('Average Validation Accuracy')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# ADD VALUE LABELS
for i, v in enumerate(depth_data['Validation Accuracy']):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'depth_impact.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'depth_impact.pdf'), dpi=300)
plt.show()

# 4. IMPACT OF REGULARIZATION
plt.figure(figsize=(12, 6))
reg_data = results_df.groupby('Regularization')['Validation Accuracy'].mean().reset_index()
reg_data = reg_data.sort_values('Validation Accuracy', ascending=False)

plt.bar(reg_data['Regularization'], reg_data['Validation Accuracy'], color='firebrick')
plt.title('Average Validation Accuracy by Regularization')
plt.xlabel('Regularization')
plt.ylabel('Average Validation Accuracy')
plt.ylim(0, 1.0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# ADD VALUE LABELS
for i, v in enumerate(reg_data['Validation Accuracy']):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'regularization_impact.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'regularization_impact.pdf'), dpi=300)
plt.show()

# Step 11: Analyzing Interaction Effects


**Purpose**: Examine how different factors interact with each other

**Why**: Reveals complex relationships between parameters that affect model performance


In [ ]:
import seaborn as sns

# CREATING A HEATMAP OF ARCHITECTURE VS. REGULARIZATION
plt.figure(figsize=(12, 8))
heatmap_data = results_df.pivot_table(
    values='Validation Accuracy',
    index='Architecture',
    columns='Regularization',
    aggfunc='mean'
)
sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt='.3f', vmin=0.5, vmax=1.0)
plt.title('Interaction: Architecture vs. Regularization')
plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'arch_reg_interaction.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'arch_reg_interaction.pdf'), dpi=300)
plt.show()

# CREATING A HEATMAP OF ARCHITECTURE VS. FILTER SIZE
plt.figure(figsize=(12, 8))
heatmap_data = results_df.pivot_table(
    values='Validation Accuracy',
    index='Architecture',
    columns='Filter Size',
    aggfunc='mean'
)
sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt='.3f', vmin=0.5, vmax=1.0)
plt.title('Interaction: Architecture vs. Filter Size')
plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'arch_filter_interaction.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'arch_filter_interaction.pdf'), dpi=300)
plt.show()

# CREATING A HEATMAP OF DEPTH VS. REGULARIZATION
plt.figure(figsize=(12, 8))
heatmap_data = results_df.pivot_table(
    values='Validation Accuracy',
    index='Depth',
    columns='Regularization',
    aggfunc='mean'
)
sns.heatmap(heatmap_data, annot=True, cmap='viridis', fmt='.3f', vmin=0.5, vmax=1.0)
plt.title('Interaction: Network Depth vs. Regularization')
plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'depth_reg_interaction.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'depth_reg_interaction.pdf'), dpi=300)
plt.show()

# Step 12: Evaluating the Best Model in Detail

**Purpose**: Perform a comprehensive evaluation of the best model configuration

**Why**: Provides detailed performance metrics for the optimal waste classification model


In [ ]:
# GET THE BEST MODEL CONFIGURATION
best_config = results_df.iloc[0]
best_model_name = best_config['Model Name']
print(f"Evaluating best model: {best_model_name}")

# FIND THE BEST MODEL IN OUR RESULTS
best_model = None
for result in factorial_results:
    if result['model_name'] == best_model_name:
        best_model = result['model']
        break

if best_model:
    # GENERATE PREDICTIONS FOR VALIDATION SET
    y_true = []
    y_pred = []
    for images, labels in val_ds:
        preds = best_model.predict(images)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    # CREATE CONFUSION MATRIX
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - Best Model ({best_model_name})')
    plt.tight_layout()
    plt.savefig(os.path.join(Files_save_path, 'best_model_confusion_matrix.png'), dpi=300)
    plt.savefig(os.path.join(Files_save_path, 'best_model_confusion_matrix.pdf'), dpi=300)
    plt.show()

    # GENERATE CLASSIFICATION REPORT
    report = classification_report(y_true, y_pred, target_names=class_names)
    print("\nClassification Report:")
    print(report)

    # SAVE CLASSIFICATION REPORT TO FILE
    with open(os.path.join(Files_save_path, 'best_model_classification_report.txt'), 'w') as f:
        f.write(f"Best Model: {best_model_name}\n\n")
        f.write(report)

    # CREATE BAR CHART FOR PRECISION, RECALL, AND F1-SCORE
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, labels=range(len(class_names)))

    x = np.arange(len(class_names))
    width = 0.25

    plt.figure(figsize=(14, 6))
    plt.bar(x - width, precision, width, label='Precision')
    plt.bar(x, recall, width, label='Recall')
    plt.bar(x + width, f1, width, label='F1-score')

    plt.ylabel('Score')
    plt.title(f'Classification Metrics - Best Model ({best_model_name})')
    plt.xticks(x, class_names, rotation=45, ha='right')
    plt.ylim([0, 1.1])
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    plt.savefig(os.path.join(Files_save_path, 'best_model_metrics.png'), dpi=300)
    plt.savefig(os.path.join(Files_save_path, 'best_model_metrics.pdf'), dpi=300)
    plt.show()

    # TEST ON SAMPLE IMAGES
    print("\nTesting best model on sample images:")

    # FUNCTION TO PREDICT AND VISUALIZE RESULTS
    def predict_and_visualize(model, img_path):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = img_array / 255.0  # NORMALIZE
        img_array = tf.expand_dims(img_array, 0)  # ADD BATCH DIMENSION

        # MAKING PREDICTION
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class]

        # DISPLAYING IMAGE WITH PREDICTION
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title(f"Predicted: {class_names[predicted_class]}\nConfidence: {confidence:.2f}")
        plt.axis('off')

        # GETTING THE FILENAME FROM THE PATH
        filename = os.path.basename(img_path)

        # SAVING THE VISUALIZATION
        plt.savefig(os.path.join(Files_save_path, f'prediction_{filename}'), dpi=300)
        plt.show()

        return class_names[predicted_class], confidence

    # FINDING ONE SAMPLE IMAGE FROM EACH CLASS
    sample_images = []
    for class_name in class_names:
        class_path = os.path.join(dataset_path, class_name)
        # GETTING THE FIRST IMAGE IN THE DIRECTORY
        for file in os.listdir(class_path)[:1]:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                sample_images.append(os.path.join(class_path, file))

    # PREDICTING ON EACH SAMPLE IMAGE
    results = []
    for img_path in sample_images:
        class_name = os.path.basename(os.path.dirname(img_path))
        print(f"\nTesting image from class: {class_name}")
        predicted_class, confidence = predict_and_visualize(best_model, img_path)
        results.append({
            'Image': os.path.basename(img_path),
            'True Class': class_name,
            'Predicted Class': predicted_class,
            'Confidence': confidence,
            'Correct': class_name == predicted_class
        })

    # DISPLAY RESULTS IN A TABLE
    results_df = pd.DataFrame(results)
    display(results_df)
else:
    print("Best model not found in results.")

# Step 13: Analyzing Main Effects and Interactions


**Purpose**: Perform statistical analysis of how each factor affects model performance

**Why**: Provides quantitative evidence for answering research question 3b



In [ ]:
print("Available variables:", [var for var in dir() if not var.startswith('_')])

if 'factorial_results' in locals():
    results_df = pd.DataFrame([
        {
            'Architecture': r['architecture'],
            'Filter Size': r['filter_size'],
            'Depth': r['depth'],
            'Regularization': r['regularization'],
            'Validation Accuracy': r['val_accuracy'],
            'Validation Loss': r['val_loss']
        }
        for r in factorial_results
    ])

    print("Created results_df with columns:", results_df.columns.tolist())
    print(results_df.head())

# CALCULATING MAIN EFFECTS
print("Main Effects Analysis:")
print("=====================")

# ARCHITECTURE EFFECT
arch_effect = results_df.groupby('Architecture')['Validation Accuracy'].agg(['mean', 'std', 'count'])
arch_effect = arch_effect.sort_values('mean', ascending=False)
print("\nArchitecture Effect:")
print(arch_effect)

# FILTER SIZE EFFECT
filter_effect = results_df.groupby('Filter Size')['Validation Accuracy'].agg(['mean', 'std', 'count'])
filter_effect = filter_effect.sort_values('mean', ascending=False)
print("\nFilter Size Effect:")
print(filter_effect)

# DEPTH EFFECT
depth_effect = results_df.groupby('Depth')['Validation Accuracy'].agg(['mean', 'std', 'count'])
depth_effect = depth_effect.sort_values('mean', ascending=False)
print("\nDepth Effect:")
print(depth_effect)

# REGULARIZATION EFFECT
reg_effect = results_df.groupby('Regularization')['Validation Accuracy'].agg(['mean', 'std', 'count'])
reg_effect = reg_effect.sort_values('mean', ascending=False)
print("\nRegularization Effect:")
print(reg_effect)

# CALCULATING TWO-WAY INTERACTION EFFECTS
print("\nTwo-way Interaction Effects:")
print("===========================")

# ARCHITECTURE X FILTER SIZE
arch_filter_effect = results_df.pivot_table(
    values='Validation Accuracy',
    index='Architecture',
    columns='Filter Size',
    aggfunc='mean'
)
print("\nArchitecture x Filter Size:")
print(arch_filter_effect)

# ARCHITECTURE X DEPTH
arch_depth_effect = results_df.pivot_table(
    values='Validation Accuracy',
    index='Architecture',
    columns='Depth',
    aggfunc='mean'
)
print("\nArchitecture x Depth:")

print(arch_depth_effect)

# ARCHITECTURE X REGULARIZATION
arch_reg_effect = results_df.pivot_table(
    values='Validation Accuracy',
    index='Architecture',
    columns='Regularization',
    aggfunc='mean'
)
print("\nArchitecture x Regularization:")
print(arch_reg_effect)

# SAVE THE ANALYSIS TO A TEXT FILE
with open(os.path.join(Files_save_path, 'factorial_analysis.txt'), 'w') as f:
    f.write("Main Effects Analysis:\n")
    f.write("=====================\n\n")

    f.write("Architecture Effect:\n")
    f.write(str(arch_effect) + "\n\n")

    f.write("Filter Size Effect:\n")
    f.write(str(filter_effect) + "\n\n")

    f.write("Depth Effect:\n")
    f.write(str(depth_effect) + "\n\n")

    f.write("Regularization Effect:\n")
    f.write(str(reg_effect) + "\n\n")

    f.write("Two-way Interaction Effects:\n")
    f.write("===========================\n\n")

    f.write("Architecture x Filter Size:\n")
    f.write(str(arch_filter_effect) + "\n\n")

    f.write("Architecture x Depth:\n")
    f.write(str(arch_depth_effect) + "\n\n")

    f.write("Architecture x Regularization:\n")
    f.write(str(arch_reg_effect) + "\n\n")

print(f"Analysis saved to {os.path.join(Files_save_path, 'factorial_analysis.txt')}")

# Step 14: Generating Conclusions and Recommendations


**Purpose**: Synthesize findings into actionable recommendations for waste classification models

**Why**: Directly answers both research questions with evidence-based conclusions



In [ ]:
# IDENTIFYING THE BEST CONFIGURATION FOR EACH FACTOR
best_arch = arch_effect.index[0]
best_filter = filter_effect.index[0]
best_depth = depth_effect.index[0]
best_reg = reg_effect.index[0]

# CREATING A MARKDOWN SUMMARY OF FINDINGS
conclusions = f"""
# Full Factorial Analysis: CNN Configurations for Waste Classification

## Research Question 3a: Best Configuration
The optimal configuration for training a waste classification model using CNNs is:

- **Architecture**: {best_arch}
- **Filter Size**: {best_filter}
- **Network Depth**: {best_depth}
- **Regularization**: {best_reg}

This configuration achieved a validation accuracy of {best_config['Validation Accuracy']:.4f}.

## Research Question 3b: Impact of Different Configurations

### Architecture Impact
- {best_arch} architecture performed best with an average accuracy of {arch_effect.loc[best_arch, 'mean']:.4f}.
- The architecture choice had {'the most' if (arch_effect['mean'].max() - arch_effect['mean'].min()) > (filter_effect['mean'].max() - filter_effect['mean'].min()) and (arch_effect['mean'].max() - arch_effect['mean'].min()) > (depth_effect['mean'].max() - depth_effect['mean'].min()) and (arch_effect['mean'].max() - arch_effect['mean'].min()) > (reg_effect['mean'].max() - reg_effect['mean'].min()) else 'a significant'} impact on model performance.
- {best_arch} likely performed best because {'of its skip connections that help with gradient flow' if best_arch == 'ResNet' else 'of its efficient depthwise separable convolutions' if best_arch == 'MobileNet' else 'it provides a good balance of depth and width' if best_arch == 'VGG' else 'of its simplicity and fewer parameters'}.

### Filter Size Impact
- {best_filter} filters performed best with an average accuracy of {filter_effect.loc[best_filter, 'mean']:.4f}.
- {'Smaller filters captured fine details better' if '3x3' in best_filter else 'Medium-sized filters balanced detail and context well' if '5x5' in best_filter else 'Larger filters captured broader patterns effectively'}.
- The difference between the best and worst filter size was {filter_effect['mean'].max() - filter_effect['mean'].min():.4f}.

### Network Depth Impact
- A depth of {best_depth} layers performed best with an average accuracy of {depth_effect.loc[best_depth, 'mean']:.4f}.
- {'Deeper networks provided better feature extraction capabilities' if best_depth > 3 else 'Moderate depth provided a good balance between capacity and overfitting' if best_depth == 3 else 'Shallower networks were sufficient for this task'}.
- The difference between the best and worst depth was {depth_effect['mean'].max() - depth_effect['mean'].min():.4f}.

### Regularization Impact
- {best_reg} regularization performed best with an average accuracy of {reg_effect.loc[best_reg, 'mean']:.4f}.
- {'Batch normalization helped stabilize training and improved generalization' if best_reg == 'BatchNorm' else 'Dropout effectively prevented overfitting' if best_reg == 'Dropout' else 'L2 regularization constrained weights appropriately' if best_reg == 'L2' else 'The model did not require regularization for this dataset'}.
- The difference between the best and worst regularization was {reg_effect['mean'].max() - reg_effect['mean'].min():.4f}.

### Interaction Effects
- The combination of {best_arch} architecture with {best_filter} filters showed particularly strong performance.
- {best_arch} architecture worked especially well with {best_reg} regularization.
- Network depth of {best_depth} was most effective when combined with {best_reg} regularization.

## Recommendations for Waste Classification Models
1. **Use {best_arch} architecture** as the foundation for waste classification models.
2. **Implement {best_filter} convolutional filters** for optimal feature extraction.
3. **Design networks with {best_depth} convolutional blocks** for the right balance of capacity and efficiency.
4. **Apply {best_reg} regularization** to improve generalization.
5. **Consider interaction effects** when designing models, as certain combinations of factors work particularly well together.

## Practical Applications
This optimal configuration can be directly applied to waste classification systems in recycling facilities, smart bins, and waste management applications. The model achieves high accuracy while maintaining reasonable computational requirements.
"""

# DISPLAY THE CONCLUSIONS
display(Markdown(conclusions))

# SAVE THE CONCLUSIONS TO A FILE
with open(os.path.join(Files_save_path, 'factorial_conclusions.md'), 'w') as f:
    f.write(conclusions)

print(f"Conclusions saved to {os.path.join(Files_save_path, 'factorial_conclusions.md')}")

# Step 15: Creating a Final Optimal Model


**Purpose**: Implement the best configuration from the factorial experiment

**Why**: Provides a ready-to-use model that represents the optimal waste classification solution


In [ ]:
# CREATING THE OPTIMAL MODEL BASED ON OUR FINDINGS
optimal_architecture = best_arch
optimal_filter_size = tuple(map(int, best_filter.split('x')))
optimal_depth = best_depth
optimal_regularization = best_reg

print(f"Creating optimal model with:")
print(f"- Architecture: {optimal_architecture}")
print(f"- Filter Size: {optimal_filter_size}")
print(f"- Depth: {optimal_depth}")
print(f"- Regularization: {optimal_regularization}")

# CREATING THE MODEL
optimal_model = create_model_with_combination(
    optimal_architecture,
    optimal_filter_size,
    optimal_depth,
    optimal_regularization
)

# COMPILING THE MODEL
optimal_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# DISPLAYING MODEL SUMMARY
optimal_model.summary()

# TRAINING THE FINAL MODEL WITH MORE EPOCHS FOR BEST PERFORMANCE
final_history = optimal_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,  # MORE EPOCHS FOR FINAL MODEL
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=7, restore_best_weights=True
        )
    ]
)

# EVALUATING THE FINAL MODEL
final_val_loss, final_val_acc = optimal_model.evaluate(val_ds)
print(f"\nFinal Optimal Model:")
print(f"Validation Accuracy: {final_val_acc:.4f}")
print(f"Validation Loss: {final_val_loss:.4f}")

# SAVING THE FINAL MODEL
final_model_path = os.path.join(Files_save_path, 'optimal_waste_classification_model.keras')
optimal_model.save(final_model_path)
print(f"Final optimal model saved to {final_model_path}")

# PLOTTING LEARNING CURVES
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(final_history.history['accuracy'], label='Training Accuracy')
plt.plot(final_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Curves - Optimal Model')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(final_history.history['loss'], label='Training Loss')
plt.plot(final_history.history['val_loss'], label='Validation Loss')
plt.title('Loss Curves - Optimal Model')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'optimal_model_learning_curves.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'optimal_model_learning_curves.pdf'), dpi=300)
plt.show()